In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
img_orig = cv2.imread('Fotos 1-11-097.jpeg', cv2.COLOR_BGR2RGB) 
img_orig = cv2.cvtColor(img_orig, cv2.COLOR_RGB2BGR)


In [ ]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
img_orig_b = increase_brightness(img_orig, value=150)
img_grey  = cv2.cvtColor(img_orig_b, cv2.COLOR_BGR2GRAY)
plt.figure(figsize = (200,20))
plt.imshow(img_grey) 

In [ ]:
img_ = cv2.GaussianBlur(img_grey, (3,3), 0)
_, img_thr = cv2.threshold(img_, 230, 255, cv2.THRESH_BINARY)

#img_thr = cv2.adaptiveThreshold(img_, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, 5)
#_, img_thr = cv2.threshold(img_, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

plt.figure(figsize = (200,20))
plt.imshow(img_thr, cmap="gray") 

In [ ]:
# erossion and dilatation

kernel = np.ones((3,3), np.uint8)

img_erodil = img_thr

img_erodil = cv2.dilate(img_erodil, kernel, iterations = 10)
img_erodil = cv2.erode(img_erodil, kernel, iterations = 10)


plt.figure(figsize = (200,20))
plt.imshow(img_erodil, cmap="gray") 

In [ ]:
img_erodil = cv2.bitwise_not(img_erodil)

In [ ]:
# nofunciona
#t  = 18
t  = 10
cont = 0
contornos, _ = cv2.findContours(img_erodil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(len(contornos))
for c in contornos:
    if len(c)>t: 
        #print(len(c))
        cont = cont + 1

print("cant contornos", cont)

In [ ]:
#img_fin = img_orig

ds = 55

img_fin = img_erodil.copy()

img_fin2 = img_orig.copy()

hmin = 20
wmin = 20
cont = 0

hmax = 120
wmax = 120
rect = 150

y_text = 10

font = cv2.FONT_HERSHEY_SIMPLEX

for c in contornos:
    if len(c)>t:
        (x, y, w, h) = cv2.boundingRect(c)
        x0 = int((x + w/2) - (rect/2))
        y0 = int((y + h/2) - (rect/2))
        cont=cont+1


        if ((h>hmin) and (w>wmin)) and ((h<hmax) and (w<wmax)):

            q = 0
            for x in range(x0, x0+rect):
                for y in range(y0, y0+rect):
                    if(img_erodil[y, x]>0): 
                        q = q + 1
                        if (cont==ds): 
                            img_fin[y, x]=100
                            

                    
            
            p = round((q/(150*150))*100, 1)
            print('OBJ #', cont, ' ', p, ' %')

            if (p>4) and (p<20):
                #cv2.rectangle(img_thr, (x0, y0), (x0+rect, y0+rect), (255, 255, 255), 5)
                cv2.rectangle(img_fin, (x0, y0), (x0+rect, y0+rect), (255, 0, 255), 5)
                cv2.rectangle(img_fin2, (x0, y0), (x0+rect, y0+rect), (255, 0, 255), 5)
                
                cv2.putText(img_fin, str(cont), (x0, y0-y_text), font, 1, (255, 0, 255), 3, cv2.LINE_AA)
                cv2.putText(img_fin2, str(cont), (x0, y0-y_text), font, 1, (255, 0, 255), 3, cv2.LINE_AA)


                # UMBRAL MINIMO 4%
                # UMBRAL MAXIMO 20%
                # cv2.rectangle(img_fin2, (x0+rect, y0+rect), (x0+rect-100, y0+rect-30), (255, 255, 255), -1)
                cv2.rectangle(img_fin, (x0+rect, y0+rect), (x0+rect-100, y0+rect-30), (255, 255, 255), -1)
                cv2.putText(img_fin, str(p)+" %" , (x0+rect-95, y0+rect-5), font, 1, (100, 100, 100), 2, cv2.LINE_AA)

                
                roi = img_orig[y0:y0+rect, x0:x0+rect]
                cv2.imwrite(str(cont)+'.png', roi)

            
        else:
            
            cv2.rectangle(img_fin, (x0, y0), (x0+rect, y0+rect), (0, 0, 255), 5)
            cv2.rectangle(img_fin2, (x0, y0), (x0+rect, y0+rect), (0, 255, 255), 5)

            cv2.putText(img_fin, str(cont), (x0, y0-y_text), font, 1, (0, 255, 255), 3, cv2.LINE_AA)
            cv2.putText(img_fin2, str(cont), (x0, y0-y_text), font, 1, (0, 255, 255), 3, cv2.LINE_AA)
        

print(cont)

In [ ]:
plt.figure(figsize = (5,5))
plt.imshow(roi)



In [ ]:
plt.figure(figsize = (200,20))
plt.imshow(img_fin)

In [ ]:
plt.figure(figsize = (200,20))
plt.imshow(img_fin2)

In [ ]:
# marca el centro de masa
def mark_cdm (i, i2, x0, y0): # i porción de imagen RGB 
    
    c = 15 # la cruz tiene 2*C de lado

    #imprime la imagen
    #plt.figure(figsize = (200,30))
    #plt.imshow(i) 

    eyes_lo=np.array([90,60,60])
    eyes_hi=np.array([125,85,85])

    mask=cv2.inRange(i,eyes_lo,eyes_hi)

    #imprime la mascara
    #plt.figure(figsize = (200,30))
    #plt.imshow(mask) 

    M = cv2.moments(mask)
 
    # calculate x,y coordinate of center
    x = int(M["m10"] / M["m00"])
    y = int(M["m01"] / M["m00"])
 
    x = x0 + x
    y = y0 + y


    # put text and highlight the center
    cv2.circle(i2, (x, y), 5, (255, 255, 255), -1)
    cv2.putText(i2, "target", (x - 25, y - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
 
    cv2.line(i2, (x, y-c), (x, y+c), (255, 255, 0), 2) 
    cv2.line(i2, (x-c, y), (x+c, y), (255, 255, 0), 2) 

    # imprime imagen
    #plt.figure(figsize = (30,5))
    #plt.imshow(i) 
    

In [ ]:
rect = 150
for c in contornos:
    print(c)
    (x, y, w, h) = cv2.boundingRect(c)
    x0 = int((x + w/2) - (rect/2))
    y0 = int((y + h/2) - (rect/2))

    roi = img_orig[y0:y0+rect, x0:x0+rect]
    mark_cdm(roi, img_fin2, x0, y0)
    


In [ ]:

plt.figure(figsize = (300,100))
plt.imshow(img_fin2) 
    

solapamiento:

si el cuerpo supera una cantidad de pixels se presume 2 moscas pegadas y se marca
si dentro de un boundingbox existen dos cuerpos se presume 2 moscas pegadas y se marca


funcion de clasificacion aleatoria


funcion de TSP:


In [ ]:
# solapamiento
